In [8]:
import tkinter as tk
from tkinter.ttk import *
from PIL import ImageTk, Image
import webbrowser
import pandas as pd
import os

from tkinter import * 
import tkinter.messagebox
from tkinter import ttk
import tkinterweb
from IPython.display import Audio
from urllib.request import urlopen

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict, Counter
from tqdm import tqdm
from langdetect import detect, DetectorFactory
import multiprocessing
import os
import pickle

from search_functionality.rocchio_filter import Napster2_Rocchio_Feedback
from user_interface.user_interface_class import Napster_GUI_Object

/Users/cardoni/napster_2/.venv/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TruncatedSVD from version 1.0.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/cardoni/napster_2/.venv/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.0.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/cardoni/napster_2/.venv/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 1.0.2 when using version 1.1.2. This might lead to brea

In [6]:
!pip install pillow
!pip install tk
!pip install sklearn
!pip install spotipy
!pip install numpy
!pip install langdetect
!pip install tqdm
!pip install pandas
!pip install tkinterweb



  Using cached tkinterweb-3.14.3.tar.gz (1.1 MB)
  Preparing metadata (setup.py) ... done
Using legacy 'setup.py install' for tkinterweb, since package 'wheel' is not installed.
  Running setup.py install for tkinterweb ... done


In [9]:
!pip freeze

appnope==0.1.3
asttokens==2.0.7
async-timeout==4.0.2
backcall==0.2.0
certifi==2022.6.15
charset-normalizer==2.1.0
debugpy==1.6.2
decorator==5.1.1
Deprecated==1.2.13
entrypoints==0.4
executing==0.10.0
idna==3.3
ipykernel==6.15.1
ipython==8.4.0
jedi==0.18.1
joblib==1.1.0
jupyter-client==7.3.4
jupyter-core==4.11.1
langdetect==1.0.9
matplotlib-inline==0.1.3
nest-asyncio==1.5.5
numpy==1.23.2
packaging==21.3
pandas==1.4.3
parso==0.8.3
pexpect==4.8.0
pickleshare==0.7.5
Pillow==9.2.0
prompt-toolkit==3.0.30
psutil==5.9.1
ptyprocess==0.7.0
pure-eval==0.2.2
Pygments==2.12.0
pyparsing==3.0.9
python-dateutil==2.8.2
pytz==2022.2.1
pyzmq==23.2.1
redis==4.3.4
requests==2.28.1
scikit-learn==1.1.2
scipy==1.9.0
six==1.16.0
sklearn==0.0
spotipy==2.20.0
stack-data==0.4.0
threadpoolctl==3.1.0
tk==0.1.0
tkinterweb==3.14.3
tornado==6.2
tqdm==4.64.0
traitlets==5.3.0
urllib3==1.26.11
wcwidth==0.2.5
wrapt==1.14.1
